In [92]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, InputLayer
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanAbsoluteError, MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import product

import plotly.graph_objects as go
import matplotlib.pyplot as plt

parameters = {
    "dataset":{
        "path": "../data/Processed_Data/Demand_Dataset.csv",
        "trainingSize": .70,
        "validationSize": .15,
        "testSize": .15
    },
    "backtesting":{
        "steps": 96,
        "fixedTrainSize": False,
        "refit": False,
    },
    "validation": {
        "n_splits" : 10
    },
    "lstm":{
        "window_size": 5,
        "epochs": 10,
        "batch_size": 96,
        "n_blocks": 1
    }
}

In [93]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2194216137856051775
xla_global_id: -1
]


In [94]:
#Read df
df = pd.read_csv(parameters['dataset']['path'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70080 entries, 0 to 70079
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Date                         70080 non-null  object 
 1   Demand                       70080 non-null  int64  
 2   Temperature                  70080 non-null  float64
 3   Relative_Humidity            70080 non-null  float64
 4   Precipitation_Total          70080 non-null  float64
 5   Sunshine_Duration            70080 non-null  float64
 6   Shortwave_Radiation          70080 non-null  float64
 7   Direct_Shortwave_Radiation   70080 non-null  float64
 8   Diffuse_Shortwave_Radiation  70080 non-null  float64
 9   Year                         70080 non-null  int64  
 10  Month                        70080 non-null  int64  
 11  Hour                         70080 non-null  int64  
 12  DayOfWeek                    70080 non-null  int64  
 13  DayOfYear       

In [95]:
#Drop Columns
dates = df["Date"]
df = df.drop("Date", axis = 1)

In [96]:
#Change Data Types
def parseData(df):
    for col in df:
        df[col] = df[col].astype(float)
    return df

#Convert to float
df = parseData(df)

In [97]:
y = df["Demand"]
X = df.drop("Demand", axis = 1)
X, y = X.to_numpy(), y.to_numpy()

#Split Dataset
trainingSize = int(parameters["dataset"]["trainingSize"] * X.shape[0])
validationSize = int(parameters["dataset"]["validationSize"] * X.shape[0]) + trainingSize

X_train, y_train = X[:trainingSize], y[:trainingSize]
X_val, y_val = X[trainingSize : validationSize], y[trainingSize : validationSize]
X_test, y_test = X[validationSize:], y[validationSize:]

X_train_dates = dates.loc[:trainingSize]
X_validation_dates = dates.loc[trainingSize:validationSize]
X_test_dates = dates.loc[validationSize:]    

#Reshape Datasets
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
y_train = y_train.reshape((y_train.shape[0], 1))
y_val = y_val.reshape((y_val.shape[0], 1))
y_test = y_test.reshape((y_test.shape[0], 1))

print(f"Training ==> X Shape: {X_train.shape}, y Shape: {y_train.shape}")
print(f"Validation ==> X Shape: {X_val.shape}, y Shape: {y_val.shape}")
print(f"Test ==> X Shape: {X_test.shape}, y Shape: {y_test.shape}")

Training ==> X Shape: (49056, 20, 1), y Shape: (49056, 1)
Validation ==> X Shape: (10512, 20, 1), y Shape: (10512, 1)
Test ==> X Shape: (10512, 20, 1), y Shape: (10512, 1)


In [98]:
#Model
def get_LSTM_Model(num_features):
    model = Sequential()
    model.add(InputLayer((num_features, 1)))
    model.add(LSTM(64, activation='tanh'))
    model.add(Dense(8, 'relu'))
    model.add(Dense(1, 'linear'))
    return model
model = get_LSTM_Model(X_test.shape[1])
model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_9 (LSTM)                   │ (None, 64)             │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 8)              │           520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,425 (68.07 KB)

 Trainable params: 17,425 (68.07 KB)

 Non-trainable params: 0 (0.00 B)

In [99]:
cp = ModelCheckpoint('model.keras', save_best_only=True, monitor='val_root_mean_squared_error', mode='max', verbose=False)
model.compile(
    loss=MeanSquaredError(), 
    optimizer=Adam(learning_rate=0.0001), 
    metrics=[
        MeanAbsoluteError(),
        RootMeanSquaredError()]
)

In [101]:
train_history = model.fit(
    X_train, 
    y_train, 
    validation_data=(X_val, y_val), 
    epochs=5,
    batch_size = parameters["lstm"]["batch_size"],
    callbacks=[cp]
)

Epoch 1/5
511/511 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 2549.1123 - mean_absolute_error: 35.8495 - root_mean_squared_error: 50.4859 - val_loss: 1880.0684 - val_mean_absolute_error: 30.1000 - val_root_mean_squared_error: 43.3598
Epoch 2/5
511/511 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 2262.0652 - mean_absolute_error: 33.3776 - root_mean_squared_error: 47.5560 - val_loss: 1701.7024 - val_mean_absolute_error: 28.5973 - val_root_mean_squared_error: 41.2517
Epoch 3/5
511/511 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 2002.9430 - mean_absolute_error: 31.3779 - root_mean_squared_error: 44.7510 - val_loss: 1656.2694 - val_mean_absolute_error: 29.3304 - val_root_mean_squared_error: 40.6973
Epoch 4/5
511/511 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 1855.7638 - mean_absolute_error: 30.6366 - root_mean_squared_error: 43.0772 - val_loss: 1619.5261 - val_mean_absolute_error: 28.8079 - val_root_mean_squared_error: 40.2433
Epoch 5/5
511/511 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 1724.7766 - mean_

In [102]:
test_history = model.evaluate(
    x = X_test,
    y = y_test,
    batch_size = parameters["lstm"]["batch_size"],
)
predictions = model.predict(
    x = X_test,
    batch_size = parameters["lstm"]["batch_size"]
)

110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1141.1196 - mean_absolute_error: 22.5947 - root_mean_squared_error: 33.3945
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


In [103]:
def plotPredictions(dates, y_pred, y_test):
    fig = go.Figure()
    trace1 = go.Scatter(x=dates, y=y_test, name="test", mode="lines")
    trace2 = go.Scatter(x=dates, y=y_pred, name="predictions", mode="lines")
    fig.add_trace(trace1)
    fig.add_trace(trace2)
    fig.update_layout(
        title="Real value vs Predicted in Test Data",
        xaxis_title="Date Time",
        yaxis_title="Demand",
        width=1020,
        height=450,
        margin = dict(l=70, r=20, t=55, b=20),
        legend = dict(
            orientation = "h",
            yanchor="top",
            y=1.1,
            xanchor="left",
            x=0.76
        )
    )
    fig.show()

plotPredictions(X_test_dates, predictions.flatten(), y_test.flatten())

In [104]:
def plotLosses(train_loss, validation_loss):
    epochs_list = np.arange(0, 50)
    fig = go.Figure()
    trace1 = go.Scatter(x=epochs_list, y=train_loss, name="Training Loss", mode="lines")
    trace2 = go.Scatter(x=epochs_list, y=validation_loss, name="Validation Loss", mode="lines")
    fig.add_trace(trace1)
    fig.add_trace(trace2)
    fig.update_layout(
        title = "Training and Validation Losses",
        xaxis_title = "MSE Loss Value",
        yaxis_title = "Epoch",
        width = 800,
        height = 400,
        margin = dict(l=40, r=30, t=50, b=30),
        legend = dict(
            orientation = "h",
            yanchor = "top",
            y = .98,
            xanchor = "right",
            x = .98
        ) 
    )
    fig.show()

plotLosses(train_history.history["loss"], train_history.history["val_loss"])

In [105]:
def plotMetrics(train_rmse, validation_rmse, train_mae, validation_mae):
    epochs_list = np.arange(0, 50)
    fig = go.Figure()
    trace1 = go.Scatter(x=epochs_list, y=train_rmse, name="Training RMSE", mode="lines")
    trace2 = go.Scatter(x=epochs_list, y=validation_rmse, name="Validation RMSE", mode="lines")
    trace3 = go.Scatter(x=epochs_list, y=train_mae, name="Training MAE", mode="lines")
    trace4 = go.Scatter(x=epochs_list, y=validation_mae, name="Validation MAE", mode="lines")

    fig.add_trace(trace1)
    fig.add_trace(trace2)
    fig.add_trace(trace3)
    fig.add_trace(trace4)
    fig.update_layout(
        title = "Metrics in Training/Validation",
        xaxis_title = "RMSE / MAE",
        yaxis_title = "Epoch",
        width = 800,
        height = 400,
        margin = dict(l=40, r=30, t=50, b=30),
        legend = dict(
            orientation = "h",
            yanchor = "top",
            y = .98,
            xanchor = "right",
            x = .98
        ) 
    )
    fig.show()
plotMetrics(train_history.history["root_mean_squared_error"], train_history.history["val_root_mean_squared_error"], train_history.history["mean_absolute_error"], train_history.history["val_mean_absolute_error"])

In [108]:
def gridSearch(data, lr_params, batch_params):
    X_train, y_train = data["train"]["X"], data["train"]["y"]
    X_val, y_val = data["validation"]["X"], data["validation"]["y"]
    dict1 = {"Model": [ ], "Batch_Size": [], "Learning_Rate": [], "Train_MAE": [], "Train_RMSE": [], "Val_MAE": [], "Val_RMSE": []}
    results = pd.DataFrame(dict1)

    for lr, batch_s in tqdm(product(lr_params, batch_params), total=len(batch_params)*len(lr_params)): 
        model = get_LSTM_Model(X_train.shape[1])
        cp = ModelCheckpoint('model.keras', save_best_only=True, monitor='val_root_mean_squared_error', mode='max', verbose=False)
        model.compile(
            loss = MeanSquaredError(), 
            optimizer=Adam(learning_rate=lr), 
            metrics=[
                MeanAbsoluteError(),
                RootMeanSquaredError()]
        )
        train_history = model.fit(
            X_train, 
            y_train, 
            validation_data=(X_val, y_val), 
            epochs=5,
            batch_size = batch_s,
            callbacks=[cp],
            verbose=False
        )
        train_mae = train_history.history["mean_absolute_error"][-1]
        train_rmse = train_history.history["root_mean_squared_error"][-1]
        val_mae = train_history.history["val_mean_absolute_error"][-1]
        val_rmse = train_history.history["val_root_mean_squared_error"][-1]

        results.loc[len(results.index)] = ['LSTM_1_Layer', batch_s, lr, train_mae, train_rmse, val_mae, val_rmse] 
    
    return results


In [109]:
data = {
    "train" : { "X": X_train, "y": y_train },
    "validation": { "X": X_val, "y": y_val },
    "test": { "X": X_test, "y": y_test }
}
batch_size_params = [32, 64, 96, 128]
lr_params = [0.0001, 0.001, 0.01]
results = gridSearch(data, lr_params, batch_size_params)
results = results.sort_values(by=["Val_RMSE", "Val_MAE"])
results

  8%|▊         | 1/12 [00:57<10:35, 57.77s/it]c:\Users\ssanchez\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\callbacks\model_checkpoint.py:206: UserWarning:

Can save best model only with val_accuracy available, skipping.

 17%|█▋        | 2/12 [01:40<08:12, 49.21s/it]c:\Users\ssanchez\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\callbacks\model_checkpoint.py:206: UserWarning:

Can save best model only with val_accuracy available, skipping.

 25%|██▌       | 3/12 [02:21<06:48, 45.38s/it]c:\Users\ssanchez\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\callbacks\model_checkpoint.py:206: UserWarning:

Can save best model only with val_accuracy available, skipping.

 33%|███▎      | 4/12 [02:46<04:58, 37.30s/it]c:\Users\ssanchez\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\callbacks\model_checkpoint.py:206: UserWarning:

Can save best model only with val_accuracy available, skipping.

 42%|████▏     |

,Model,Batch_Size,Learning_Rate,Train_MAE,Train_RMSE,Val_MAE,Val_RMSE
9,LSTM_1_Layer,64,0.0100,22.582695,35.122066,24.694973,36.767315
10,LSTM_1_Layer,96,0.0100,23.074451,35.616955,25.466164,37.474880
5,LSTM_1_Layer,64,0.0010,21.543646,33.732624,24.657127,37.739246
4,LSTM_1_Layer,32,0.0010,21.423779,33.635384,24.720020,37.988358
8,LSTM_1_Layer,32,0.0100,22.489487,34.965042,24.319996,38.206581
6,LSTM_1_Layer,96,0.0010,22.469711,34.541718,25.337158,38.267498
7,LSTM_1_Layer,128,0.0010,23.576401,35.674919,26.865396,38.527493
0,LSTM_1_Layer,32,0.0001,26.367708,38.746212,28.247236,40.427181
2,LSTM_1_Layer,96,0.0001,30.423336,42.444828,29.611191,40.711777
3,LSTM_1_Layer,128,0.0001,31.330803,43.584171,30.111441,40.723118
